In [3]:
import pandas as pd

In [7]:
df = pd.read_csv("spotify_millsongdata.csv")

In [8]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [9]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [10]:
df.shape

(57650, 4)

In [11]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [12]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [13]:
df.head(10)

,artist,song,text
0,Beautiful South,Last Waltz,Don't feel bad if you wake up feeling suddenly...
1,Perry Como,Delaware,"Oh what did Del-a-ware boy, what did Delaware ..."
2,America,Love's Worn Out Again,"Like you said, you'd never get used to it \r\..."
3,Weird Al Yankovic,Hardware Store,Nothin' ever (ever) happens in this town \r\n...
4,Moody Blues,From The Bottom Of My Heart (I Love You),I can't be any more \r\nIn love with you \r\...
5,Lata Mangeshkar,Chanda Hai Tu,"(Chanda hai tu, mera sooraj hai tu \r\nO meri..."
6,Elvis Costello,Call On Me (Demo),Everyday seem like Sunday \r\nIt must be some...
7,Neil Diamond,"A Little Bit Me, A Little Bit You",Walk out \r\nGirl don't you walk out \r\nWe'...
8,XTC,Red,You better watch your tape boys \r\nIt's the ...
9,Lady Gaga,The Heart Interlude,"-""Where is it? Where is it, Candy? Candy?"" \r..."


In [14]:
df['text'][0]

"Don't feel bad if you wake up feeling suddenly old  \r\nMore the realisation that you've let yourself be covered by mould  \r\nAge is not something you wrap up if you're feeling the cold  \r\nIt's a laugh ten times harder at a joke eleven times told  \r\n  \r\nDown to the churchyard, check out the vaults  \r\nDig up the coffins, unscrew the bolts  \r\nNo better reminder, no strong enough salts  \r\nTo show this pitiful few that we've missed the last waltz  \r\n  \r\nDon't feel bad if you're feeling past you sell-by date  \r\nLike the roller coaster ride slowed down to a serious debate  \r\nOh the weather you tell them you're ready or whether you wait  \r\nDoes someone place a hand on your shoulder you point the way to the gate  \r\n  \r\nDown to the churchyard, check out the vaults  \r\nDig up the coffins, unscrew the bolts  \r\nNo better reminder, no strong enough salts  \r\nTo show this pitiful few that we've missed the last waltz  \r\n  \r\nYou don't count the birthdays 'til your o

In [15]:
# df = df.sample(5000)

In [16]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [17]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [18]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [19]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [22]:
similarity[0]

array([1.        , 0.01491848, 0.05931035, ..., 0.04950724, 0.01011302,
       0.03905872])

In [23]:
df[df['song'] == 'Crying Over You']

,artist,song,text
2591,UB40,Crying Over You,cri over you in the morn cri over you in the e...


In [24]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [25]:
recommendation('Crying Over You')

['All Cried Out',
 "Don't Cry",
 'Cry No More',
 'Cry Me A River',
 'My Heart Cries For You',
 "I'll Be There",
 'True Love Always',
 'Crying Time',
 "I've Got A Right To Cry",
 'Now And Then',
 "I Ain't Gonna Cry Tonight",
 'A Lovely Place To Cry',
 'You Made Me Love You',
 'I Wish It Would Rain',
 'I Cried Again',
 'Blue Eyes Crying In The Rain',
 "Don't Explain",
 "Don't Explain",
 'I Let The Music Speak',
 'The Boy In The Bubble']

In [26]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))